<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 15:38:06
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: 11.56 K (0.08%)
Current PnL: -31.20 L (-19.5%)
CY Booked + Current PnL: -15.52 L (-9.7%)
-------------------
Total profit:  1.43 L
Total loss:  -32.62 L
-------------------
Total Booked + Current PnL: 11.46 L (8.62%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.95%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.54 L (67.4%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-0.24,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,59.0,H-SC,4.88,167.0,0.17,0.73,47.54,XR,ATH,FINANCE
18,COALINDIA,0.47,2.79,14.48,17.67,22395.0,4200.0,154660.0,478.51,23.39,50.0,L-LC,11.37,185.0,0.19,1.12,24.15,XY25,ATH,MINING
77,TTKPRESTIG,-0.67,-27.80,38.63,0.09,28109.0,-28013.0,72764.0,770.00,68.84,38.0,M-SC,9.38,250.0,-1.00,0.53,0.14,OX40N,NTT,DURABLES
36,ICICIGI,1.63,3.81,14.96,19.34,31076.0,7619.0,207728.0,2252.93,-12.72,65.0,X-MC,2.82,66.0,0.25,1.50,20.35,X40,ATH,INSURANCE
51,MEDANTA,0.06,-2.05,26.67,24.08,33164.0,-2601.0,124349.0,1486.00,-2.66,58.0,X-SC,11.02,87.0,-0.08,0.90,13.89,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COLPAL,2.46,-16.32,69.10,41.51,152289.0,-42975.0,220390.0,3726.84,1.32,62.0,X-MC,6.21,62.0,-0.28,1.59,7.69,XY25,ATH,FMCG
49,LAOPALA,2.27,-41.04,135.92,39.09,105059.0,-53810.0,77295.0,464.00,69.11,47.0,H-SC,15.30,154.0,-0.51,0.56,3.36,AR,NTT,CERAMICS
44,ITC,2.15,-18.52,35.96,10.78,70414.0,-44505.0,195813.0,452.00,-51.88,55.0,X-LC,18.65,1.0,-0.63,1.42,7.85,X40,NTT,FMCG
11,BANDHANBNK,1.97,-17.38,132.91,92.42,305868.0,-48427.0,230132.0,400.00,97.56,71.0,H-SC,7.03,174.0,-0.16,1.66,33.00,XY24,NTT,BANKS
80,VALIANTORG,1.92,-48.80,240.15,74.15,201154.0,-79843.0,83762.0,838.00,-250.97,43.0,H-SC,24.09,145.0,-0.40,0.61,7.37,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-2.60,-31.87,121.31,50.79,116250.0,-44824.0,95829.0,831.70,-12.64,36.0,H-SC,7.00,172.0,-0.39,0.69,11.58,XR,ATH,IT
61,ROUTE,-2.05,-59.84,318.72,68.16,170321.0,-79627.0,53439.0,2215.45,-67.37,27.0,H-SC,28.15,141.0,-0.47,0.39,0.00,SR,ATH,IT
56,RAJOOENG,-2.00,-36.37,119.41,39.61,77879.0,-37280.0,65220.0,143.10,-56.99,49.0,H-SC,28.64,136.0,-0.48,0.47,9.26,AR,ATH,MISC
39,INDIAMART,-1.99,-6.34,120.72,106.72,139449.0,-7822.0,115514.0,4810.62,-57.81,44.0,H-SC,7.80,140.0,-0.06,0.84,15.55,AR,ATH,MISC
19,COFFEEDAY,-1.90,-44.08,141.91,35.27,90104.0,-50055.0,63494.0,80.00,-57.75,52.0,L-SC,14.43,266.0,-0.56,0.46,55.40,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.15,3.85,100.21,107.91,153643.0,5677.0,153321.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.35,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.67,-27.80,38.63,0.09,28109.0,-28013.0,72764.0,770.00,68.84,38.0,M-SC,9.38,250.0,-1.00,0.53,0.14,OX40N,NTT,DURABLES
66,SIS,0.42,-28.56,69.75,21.26,55324.0,-31714.0,79318.0,528.00,1865.28,37.0,H-SC,4.23,168.0,-0.57,0.57,9.35,OX40N,NTT,MISC
72,TATAELXSI,-0.69,-29.60,87.53,32.03,81241.0,-39020.0,92815.0,9161.00,-11.96,36.0,H-SC,7.23,158.0,-0.48,0.67,3.96,OX40N,NTT,IT
40,INDIGOPNTS,-0.36,-30.78,44.48,0.02,53749.0,-53721.0,120838.0,1408.00,84.15,30.0,M-SC,8.68,222.0,-1.00,0.87,5.11,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.9,20.74,20.25,45.2,46881.0,39768.0,231510.0,1856.0,38.21,68.0,X-MC,4.51,79.0,0.85,1.67,28.95,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.47,2.79,14.48,17.67,22395.0,4200.0,154660.0,478.51,23.39,50.0,L-LC,11.37,185.0,0.19,1.12,24.15,XY25,ATH,MINING
26,FINCABLES,0.15,3.85,100.21,107.91,153643.0,5677.0,153321.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.35,OX40N,ATH,CABLES
50,MASFIN,-0.24,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,59.0,H-SC,4.88,167.0,0.17,0.73,47.54,XR,ATH,FINANCE
1,ABB,0.78,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,69.0,H-MC,4.32,125.0,0.21,2.04,25.37,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.78,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,69.0,H-MC,4.32,125.0,0.21,2.04,25.37,AR,NTT,ELECTRICAL
50,MASFIN,-0.24,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,59.0,H-SC,4.88,167.0,0.17,0.73,47.54,XR,ATH,FINANCE
26,FINCABLES,0.15,3.85,100.21,107.91,153643.0,5677.0,153321.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.35,OX40N,ATH,CABLES
39,INDIAMART,-1.99,-6.34,120.72,106.72,139449.0,-7822.0,115514.0,4810.62,-57.81,44.0,H-SC,7.80,140.0,-0.06,0.84,15.55,AR,ATH,MISC
85,ZYDUSLIFE,0.37,-4.03,41.33,35.63,83123.0,-8453.0,201121.0,1286.17,-16.67,53.0,H-MC,5.41,120.0,-0.10,1.45,11.97,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.26,-6.48,43.55,34.24,127426.0,-20290.0,292598.0,1972.00,-26.34,24.0,X-LC,3.95,3.0,-0.16,2.12,1.26,X40,NTT,IT
73,TCS,-0.83,-25.90,59.99,18.55,168134.0,-97967.0,280270.0,4311.59,-33.14,27.0,X-LC,4.21,2.0,-0.58,2.03,0.10,X40,ATH,IT
33,HCLTECH,-1.04,-4.78,30.06,23.83,69243.0,-11574.0,230350.0,1908.19,0.08,28.0,X-LC,4.55,8.0,-0.17,1.67,10.52,X40,ATH,IT
8,AWL,-0.68,-34.31,139.31,57.19,298433.0,-111905.0,214222.0,485.00,-62.47,29.0,X-SC,17.56,81.0,-0.37,1.55,0.55,XY24,NTT,FMCG
13,BERGEPAINT,-0.65,-17.98,47.70,21.14,88942.0,-40877.0,186462.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.41,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.04,4.65,15.40,20.77,37735.0,10887.0,245031.0,1644.00,-7.24,64.0,X-MC,1.46,64.0,0.29,1.77,12.40,X40N,NTT,BREWERIES
21,DABUR,-1.20,-0.17,43.95,43.70,108838.0,-427.0,247641.0,735.00,-9.49,48.0,X-MC,1.89,70.0,-0.00,1.79,14.85,XY24,BTT,FMCG
65,SIEMENS,0.05,-14.35,46.54,25.51,74182.0,-26700.0,159395.0,4671.50,33.78,56.0,H-LC,1.95,51.0,-0.36,1.15,17.46,AR,ATH,ELECTRICAL
36,ICICIGI,1.63,3.81,14.96,19.34,31076.0,7619.0,207728.0,2252.93,-12.72,65.0,X-MC,2.82,66.0,0.25,1.50,20.35,X40,ATH,INSURANCE
34,HINDUNILVR,0.48,-9.61,25.76,13.67,59853.0,-24713.0,232350.0,2922.00,-32.95,43.0,X-LC,2.86,9.0,-0.41,1.68,9.69,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-0.83,-25.90,59.99,18.55,168134.0,-97967.0,280270.0,4311.59,-33.14,27.0,X-LC,4.21,2.0,-0.58,2.03,0.10,X40,ATH,IT
53,PGHH,0.01,-13.12,51.95,32.01,96369.0,-28016.0,185504.0,17616.87,-41.22,35.0,X-MC,9.31,57.0,-0.29,1.34,0.27,X40,ATH,FMCG
13,BERGEPAINT,-0.65,-17.98,47.70,21.14,88942.0,-40877.0,186462.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.41,XY24,NTT,PAINTS
3,ACC,0.07,-31.05,138.08,64.15,226534.0,-73891.0,164060.0,3906.00,-48.05,39.0,X-SC,8.47,85.0,-0.33,1.19,0.51,XY24,BTT,CEMENT
8,AWL,-0.68,-34.31,139.31,57.19,298433.0,-111905.0,214222.0,485.00,-62.47,29.0,X-SC,17.56,81.0,-0.37,1.55,0.55,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.19,-29.92,103.84,42.84,47303.0,-19452.0,45554.0,424.00,-54.27,52.0,X-SC,18.84,80.0,-0.41,0.33,9.58,XY24,NTT,MISC
12,BATAINDIA,-0.22,-46.01,150.43,35.20,104499.0,-59203.0,69467.0,2096.00,-8.88,37.0,X-SC,21.51,91.0,-0.57,0.50,0.98,X40,NTT,FOOTWEAR
58,RELAXO,1.22,-51.74,215.58,52.31,151031.0,-75102.0,70058.0,1176.00,-48.60,48.0,X-SC,14.03,92.0,-0.50,0.51,5.03,X40N,NTT,FOOTWEAR
51,MEDANTA,0.06,-2.05,26.67,24.08,33164.0,-2601.0,124349.0,1486.00,-2.66,58.0,X-SC,11.02,87.0,-0.08,0.90,13.89,XY24,NTT,HEALTHCARE
35,HONAUT,0.00,-22.95,86.00,43.32,107930.0,-37372.0,125500.0,58357.33,-32.37,39.0,X-SC,11.16,90.0,-0.35,0.91,1.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,2.15,-18.52,35.96,10.78,70414.0,-44505.0,195813.0,452.00,-51.88,55.0,X-LC,18.65,1.0,-0.63,1.42,7.85,X40,NTT,FMCG
73,TCS,-0.83,-25.90,59.99,18.55,168134.0,-97967.0,280270.0,4311.59,-33.14,27.0,X-LC,4.21,2.0,-0.58,2.03,0.10,X40,ATH,IT
42,INFY,-1.26,-6.48,43.55,34.24,127426.0,-20290.0,292598.0,1972.00,-26.34,24.0,X-LC,3.95,3.0,-0.16,2.12,1.26,X40,NTT,IT
75,TMPV,0.00,-21.53,56.72,22.98,100542.0,-48638.0,177260.0,600.00,-81.44,64.0,X-LC,2.95,4.0,-0.48,1.28,13.32,XY24,NTT,AUTO
33,HCLTECH,-1.04,-4.78,30.06,23.83,69243.0,-11574.0,230350.0,1908.19,0.08,28.0,X-LC,4.55,8.0,-0.17,1.67,10.52,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.09,-30.26,90.55,32.89,48757.0,-23362.0,53845.0,1800.00,-353.50,65.0,L-MC,11.13,258.0,-0.48,0.39,48.29,XR,NTT,BANKS
67,SONACOMS,0.39,-8.00,51.19,39.09,47640.0,-8094.0,93065.0,804.02,-27.09,64.0,M-MC,6.70,193.0,-0.17,0.67,31.66,AR,ATH,AUTO
25,EASEMYTRIP,1.37,-46.85,174.14,45.70,163639.0,-82845.0,93970.0,26.40,-2.50,82.0,M-SC,22.97,204.0,-0.51,0.68,54.82,XY24,NTT,TRAVEL
50,MASFIN,-0.24,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,59.0,H-SC,4.88,167.0,0.17,0.73,47.54,XR,ATH,FINANCE
79,VAIBHAVGBL,-1.65,-23.76,110.86,60.77,154487.0,-43422.0,139353.0,521.00,60.88,55.0,H-SC,1.46,160.0,-0.28,1.01,26.34,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,1.37,-46.85,174.14,45.70,163639.0,-82845.0,93970.0,26.40,-2.50,82.0,M-SC,22.97,204.0,-0.51,0.68,54.82,XY24,NTT,TRAVEL
7,ATULAUTO,1.21,-10.18,62.06,45.57,109891.0,-20060.0,177072.0,844.00,3840.54,70.0,M-SC,4.39,248.0,-0.18,1.28,34.00,XY24,NTT,AUTO
1,ABB,0.78,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,69.0,H-MC,4.32,125.0,0.21,2.04,25.37,AR,NTT,ELECTRICAL
11,BANDHANBNK,1.97,-17.38,132.91,92.42,305868.0,-48427.0,230132.0,400.00,97.56,71.0,H-SC,7.03,174.0,-0.16,1.66,33.00,XY24,NTT,BANKS
67,SONACOMS,0.39,-8.00,51.19,39.09,47640.0,-8094.0,93065.0,804.02,-27.09,64.0,M-MC,6.70,193.0,-0.17,0.67,31.66,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.19
1,25,44.70
2,50,76.82


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.34
MC    30.38
LC    23.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.87
X40      23.22
X40N     15.22
AR        9.01
XY25      8.85
XR        8.70
OX40N     7.29
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.91
H-SC    22.64
X-LC    19.97
X-SC    11.87
M-SC    11.05
H-MC     4.96
M-MC     2.12
H-LC     1.15
L-LC     1.12
M-LC     1.07
L-SC     0.78
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.09,-13.48,51.61
FINANCE,13.42,-12.43,56.68
IT,11.46,-35.13,102.56
MISC,7.06,-33.20,87.69
ELECTRICAL,6.28,-8.07,47.50
PAINTS,5.08,-31.42,50.95
INSURANCE,4.89,-0.20,34.48
PHARMA,4.10,-3.73,36.38
AUTO,3.23,-17.16,57.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3255267.0,21
AR,1326014.0,10
XR,1290251.0,12
X40,1287416.0,15
X40N,1017831.0,10
OX40N,784810.0,10
XY25,392234.0,6
SR,299977.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663728.0,24
M-SC,1392401.0,14
X-MC,1345282.0,15
X-SC,1163731.0,10
X-LC,1130591.0,12
H-MC,392090.0,3
L-SC,183466.0,2
M-LC,144595.0,1
M-MC,92582.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1238067.0      6
           AR         948992.0      5
           XR         829921.0      7
M-SC       XY24       797300.0      6
X-SC       XY24       605434.0      4
X-LC       X40        577333.0      6
X-MC       X40        536195.0      7
           X40N       412137.0      4
X-SC       X40N       384409.0      4
H-SC       OX40N      346771.0      4
M-SC       OX40N      344677.0      5
H-SC       SR         299977.0      2
X-LC       X40N       221285.0      2
H-MC       XY24       210440.0      1
X-LC       XY24       206246.0      2
X-MC       XY25       199170.0      2
           XY24       197780.0      2
H-MC       AR         181650.0      2
M-SC       XR         176874.0      2
X-SC       X40        173888.0      2
M-LC       XR         144595.0      1
X-LC       XY25       125727.0      2
L-SC       OX40N       93362.0      1
           XR          90104.0      1
H-LC       AR          74182.0      1
M-SC       AR          73550.0      1
L-MC       XR          48757.0      1
M-MC       AR          47640.0      1
           XY25        44942.0      1
L-LC       XY25        22395.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
